### Group3_Project-TWO | News API
<hr />
Looking for signal in the noise using News APIsl; limited to 100 days.<br>

&nbsp;
<ul>
<font size="1" color="silver">10AUG2021</font><br>
<p>Working on some of the logic here in case we want to go down the path of displaying a recent sentiment summary for a given ticker.
</ul>
&nbsp;
<ul>
<font size="1" color="silver">14AUG2021</font><br>
<p>Made changes to the logic so that a simple "find and replace" could deliver today's sentiment on any given ticker in our S&amp;P data frame.
</ul>

In [1]:
## I just put this into everything now so I don't wast time reading the warnings.

import warnings
warnings.filterwarnings('ignore')

In [2]:
## --- Most of this is from the crypto sentiment HW.

In [3]:
## Doing some initial imports from the crypto_sentiment HW.

import os
import pandas as pd
from dotenv import load_dotenv

In [4]:
!pip install nltk

In [5]:
import nltk as nltk

## nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

In [6]:
## I had trouble getting my base environment to read my API keys.
## Trying pip install python-dotenv to make sure it can see the .env file.

!pip install python-dotenv

In [7]:
## So far, so good.

from newsapi import NewsApiClient

In [8]:
%load_ext dotenv
%dotenv

In [9]:
# Read your api key environment variable

newsapi_key = os.getenv("NEWSAPI_ORG_KEY")

In [10]:
newsapi = NewsApiClient(api_key=newsapi_key)
newsapi

In [11]:
# Fetch the relevant news articles
## Study group says we're sorting by "relevancy"; sounds right.
## Naming this one tezos_articles, because I'm using for articles about Tezos.

gm_articles = newsapi.get_everything(
    q="gm", language="en", sort_by="relevancy"
    )

In [12]:
## Let's see if that worked.
## Show some sample articles.

gm_articles["articles"][:1]

[{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
  'author': 'Kirsten Korosec',
  'title': 'GM is bringing its upgraded hands-free Super Cruise driving system to six vehicles in 2022',
  'description': 'GM is rolling out three major upgrades including automatic lanes changes and towing support to its hands-free driver assistance system Super Cruise and making it available in six vehicles, including the 2022 all-new GMC Hummer EV pickup truck. While GM has st…',
  'url': 'http://techcrunch.com/2021/07/22/gm-is-bringing-its-upgraded-hands-free-super-cruise-driving-system-to-six-vehicles-in-2022/',
  'urlToImage': 'https://techcrunch.com/wp-content/uploads/2021/07/2021-Cadillac-Escalade-Super-Cruise-109.jpg?w=600',
  'publishedAt': '2021-07-23T04:00:16Z',
  'content': 'GM is rolling out three major upgrades including automatic lanes changes and towing support to its hands-free driver assistance system Super Cruise and making it available in six vehicles, including … [+3095 chars]'}

In [13]:
## Let's make a variable to count the results.

gm_total = gm_articles["totalResults"]
print(gm_total)

4581


In [14]:
## Now I'll print an f-string (for later).
print(f"There were {gm_total} articles written about gm in our News API range.")

There were 4581 articles written about gm in our News API range.


In [15]:
## So far, so good!

In [16]:
# Creating the ticker sentiment scores DataFrame
## This came with a *TON* of help from the study group and copy/pasting from class work.

gm_sentiment = []

for article in gm_articles["articles"]:
    try:
        text = article["content"]
        sentiment = analyzer.polarity_scores(text)
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        gm_sentiment.append({
            "positive": pos,
            "negative": neg,
            "neutral": neu,
            "text": text
        })
        
    except AttributeError:
        pass

In [17]:
gm_sentiment_df = pd.DataFrame(gm_sentiment)
gm_sentiment_df.head()

,positive,negative,neutral,text
0,0.180,0.0,0.820,GM is rolling out three major upgrades includi...
1,0.000,0.0,1.000,GM will add a full-size electric pickup truck ...
2,0.000,0.0,1.000,GM is working on another electric pickup truck...
3,0.000,0.0,1.000,The twice-issued recall for 2017 to 2019 Chevr...
4,0.069,0.0,0.931,General Motors said Wednesday it is adding to ...


In [18]:
## Describe the ticker Sentiment

gm_sentiment_df.describe()

,positive,negative,neutral
count,20.000000,20.000000,20.000000
mean,0.057100,0.019400,0.923500
std,0.083139,0.045335,0.101899
min,0.000000,0.000000,0.652000
25%,0.000000,0.000000,0.861000
50%,0.000000,0.000000,0.970000
75%,0.087250,0.000000,1.000000
max,0.240000,0.158000,1.000000


In [19]:
## Converting the description to a df.

gm_described_df = gm_sentiment_df.describe()
gm_described_df

,positive,negative,neutral
count,20.000000,20.000000,20.000000
mean,0.057100,0.019400,0.923500
std,0.083139,0.045335,0.101899
min,0.000000,0.000000,0.652000
25%,0.000000,0.000000,0.861000
50%,0.000000,0.000000,0.970000
75%,0.087250,0.000000,1.000000
max,0.240000,0.158000,1.000000


In [20]:
gm_described_df['positive']['mean']

0.0571

In [21]:
## This is the logic for generating the "answers" to some of the questions.

gm_pos = round(gm_described_df['positive']['mean'],2)*100
gm_neg = round(gm_described_df['negative']['mean'],2)*100
gm_neu = round(gm_described_df['neutral']['mean'],2)*100
print(f"There were {gm_total} articles written about gm in our range; of those, {gm_pos}% of the articles were positive, {gm_neg}% were negative, and {gm_neu}% were neutral.")

There were 4581 articles written about gm in our range; of those, 6.0% of the articles were positive, 2.0% were negative, and 92.0% were neutral.


<br>
<hr />

In [22]:
## Below this line is "scratch paper".

In [23]:
## --- ## --- ## --- ##

In [24]:
## This is work I was doing to isolate the volume of chatter as articles per day.

## fomoco_articles = newsapi.get_everything(q='ford motor company',
##    from_param='2021-07-10',
##    to='2021-08-10',
##    language='en',
##  sort_by='relevancy',
##  page=1
##  )

In [25]:
## fomoco_articles['articles'][0]['publishedAt']

In [26]:
## for i in range(0,len(fomoco_articles['articles'])):
##

In [27]:
## print(fomoco_articles['articles'][i]['publishedAt'])